In [33]:
from bokeh.models import ColumnDataSource, Whisker
from bokeh.plotting import figure, show
from bokeh.sampledata.autompg2 import autompg2 as df
from bokeh.transform import factor_cmap, jitter
from bokeh.models import Range1d
import pandas as pd
import math

## Load data
file = "..\Data\cykelstativ.csv"
df = pd.read_csv(file)

df = df[df["antal_pladser"] != 0]
df["bydel"] = df["bydel"].astype(str) 
df = df[df['bydel']!= "nan"]#OBS nan er fjernet
classes = list(sorted(df["bydel"].unique()))

p = figure(height=750,width=1000, x_range=classes, background_fill_color="#efefef",
           title="Number of bike spot in each district")
p.xgrid.grid_line_color = None

g = df.groupby("bydel")
upper = g.antal_pladser.quantile(0.80)
lower = g.antal_pladser.quantile(0.20)
source = ColumnDataSource(data=dict(base=classes, upper=upper, lower=lower))

error = Whisker(base="base", upper="upper", lower="lower", source=source,
                level="annotation", line_width=1)
error.upper_head.size=20
error.lower_head.size=20
p.add_layout(error)

p.scatter(jitter("bydel", 0.3, range=p.x_range), "antal_pladser", source=df,
          alpha=0.5, size=5, line_color="white",
          color=factor_cmap("bydel", "Light7", classes))

p.xaxis.major_label_orientation = math.pi/3 #"vertical"
p.y_range = Range1d(0, 50)
show(p)